This cell is intended to process long excel 

In [ ]:
import pandas as pd

df = pd.read_excel('wines_dataset.xlsx')
#display(df.columns)
#display(df.head())

df_new = pd.read_excel("a_lot_wines_dataset.xlsx")
#display(df_new.columns)

# join food columns in 'FoodParing' column
df_new['FoodParing'] = df_new[['general_pairings','Vegetables', 'Red_meat', 'aperitifs', 'seafood', 'fish', 'pizza', 'pasta','desserts','occasion']].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)


df_new.rename(columns={'type':'Taste',"category":"Type",'code': 'ID','title':'Name','seniority':'Year',"country":"Country","volume":"Volume","price":"Price","winery_region":"Region","winery":"Winery","denomination_of_origin":"DO","variety":"GrapeVariety","vineyard_age":"VineyardAge",
                       "awarded_wines":"Award","percentage_of_alcohol":"Alcohol","temperature":"Temperature"}, inplace=True)

df_new['Link'] = df_new['Name'].apply(lambda x: f' https://disndis.com/products/{x.replace(" ","-")}?')
df_new['Allergens'] = df_new['sulphites']

#display(df[['Taste','Type']])
#display(df_new[['category','Type']])
df_new=df_new[df.columns]
df_new['Award']= df_new['Award'].apply(lambda x: True if pd.isnull(x) else False)


for col in df_new.columns:
    print(f'{col} : {df_new[col].isnull().sum()}')
    df_new[col].fillna(df_new[col].mode()[0], inplace=True)
print(df_new.isnull().sum())

df_new.to_excel('clean_wines_dataset.xlsx', index=False)

ID : 0
Name : 0
Type : 0
Year : 46
Country : 0
Region : 19
Winery : 4
Volume : 0
DO : 46
GrapeVariety : 15
VineyardAge : 372
Taste : 17
Alcohol : 0
Temperature : 12
Allergens : 28
FoodParing : 0
Price : 0
Award : 0
Link : 0
ID              0
Name            0
Type            0
Year            0
Country         0
Region          0
Winery          0
Volume          0
DO              0
GrapeVariety    0
VineyardAge     0
Taste           0
Alcohol         0
Temperature     0
Allergens       0
FoodParing      0
Price           0
Award           0
Link            0
dtype: int64


/tmp/ipykernel_10308/77201162.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_new[col].fillna(df_new[col].mode()[0], inplace=True)


This cell is intended to remove nan values from dataset

In [19]:
import pandas as pd

df = pd.read_excel('wines_dataset_2024_11_02.xlsx')
#display(df.columns)
#display(df.head())

# check for missing values in the dataset by columns
for col in df.columns:
    null_values = df[col].isnull().sum()
    if null_values > 0:
        mode = df[col].mode()[0]
        print(f'{col} : {null_values}--->replcaed by mode value:"{mode}"')
        df[col]=df[col].fillna(mode)

# check for missing values in the dataset by columns
for col in df.columns:
    null_values = df[col].isnull().sum()
    if null_values > 0:
        print(f'{col} : {null_values}')

df.to_excel('clean_wines_dataset.xlsx', index=False)

FoodParing : 33--->replcaed by mode value:"Red Meat"


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Base URL for the website
base_url = "https://disndis.com"

# List of wines to search
wines = pd.read_excel("wines_dataset_2024_11_02.xlsx")
wines = wines["Name"]

# Function to get purchase links
def get_purchase_links(wine_list):
    links = {}
    for wine in wine_list:
        # Format the search URL
        search_url = f"{base_url}/search?q={wine.replace(' ', '+')}"
        response = requests.get(search_url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find product links (update selector based on website structure)
            products = soup.select('a.product-link')  # Example selector
            
            if products:
                links[wine] = [base_url + product['href'] for product in products]
            else:
                links[wine] = "No products found"
        else:
            links[wine] = f"Error: {response.status_code}"
    return links

# Get links for the given wines
purchase_links = get_purchase_links(wines)

# Print results
for wine, link in purchase_links.items():
    print(f"{wine}: {link}")

120 Reserva Especial 2019 Chardonnay White Wine: No products found
120 Reserva Especial Syrah 2019 Red Wine: No products found
42 Zura 2018 White Wine: No products found
A.S Boires 2017 Red Wine: No products found
A.S Cava Brut Nature Gran Reserva Sello Organic 2016 Sparkling Wine: No products found
Abadia Da Cova 1124, 2016 Red Wine: No products found
Abadia Da Cova Mencia 2020 Rose Wine: No products found
Abadia Da Cova Organic 2020 White Wine: No products found
Abadia Da Cova Xuno Vina Vieja 2018 Red Wine: No products found
AD Libitum Maturana Blanca 2020 White Wine: No products found
Ad Libitum Maturana Tinta Rioja 2019 Red Wine: No products found
Alabaster 2018 Red Wine: No products found
Alboca Crianza 2018 Red Wine: No products found
Alboca Roble 2018 Red Wine: No products found
Alboca Tinto 2018 Red Wine: No products found
Aldeya Chardonnay 2019 White Wine: No products found
Aldeya Organic 2019 Rose Wine: No products found
Alion 2016 Red Wine: No products found
Almirez 2019 Red

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Configure Selenium WebDriver
driver = webdriver.Chrome()

# List of wines
# List of wines to search
wines = pd.read_excel("wines_dataset_2024_11_02.xlsx")
wines = wines["Name"]

links = {}
for wine in wines:
    search_url = f"https://disndis.com/search?q={wine.replace(' ', '+')}"
    driver.get(search_url)
    
    # Extract product links (update selector based on structure)
    try:
        product_elements = driver.find_elements(By.CSS_SELECTOR, 'a.product-link')  # Example selector
        links[wine] = [elem.get_attribute('href') for elem in product_elements]
    except Exception as e:
        links[wine] = f"Error: {e}"

# Close the browser
driver.quit()

# Print results
for wine, link in links.items():
    print(f"{wine}: {link}")


120 Reserva Especial 2019 Chardonnay White Wine: []
120 Reserva Especial Syrah 2019 Red Wine: []
42 Zura 2018 White Wine: []
A.S Boires 2017 Red Wine: []
A.S Cava Brut Nature Gran Reserva Sello Organic 2016 Sparkling Wine: []
Abadia Da Cova 1124, 2016 Red Wine: []
Abadia Da Cova Mencia 2020 Rose Wine: []
Abadia Da Cova Organic 2020 White Wine: []
Abadia Da Cova Xuno Vina Vieja 2018 Red Wine: []
AD Libitum Maturana Blanca 2020 White Wine: []
Ad Libitum Maturana Tinta Rioja 2019 Red Wine: []
Alabaster 2018 Red Wine: []
Alboca Crianza 2018 Red Wine: []
Alboca Roble 2018 Red Wine: []
Alboca Tinto 2018 Red Wine: []
Aldeya Chardonnay 2019 White Wine: []
Aldeya Organic 2019 Rose Wine: []
Alion 2016 Red Wine: []
Almirez 2019 Red Wine: []
Alonso del Yerro 2016 Red Wine: []
Alta Alella Dolc Mataro Organic 2019 Sweet Red Wine: []
Alta Alella GX Organic 2020 Red Wine: []
Alta Alella Laieta Gran Reserva Organic 2017 Sparkling Wine: []
Alta Alella Parvus Chardonnay Organic 2020 White Wine: []
Alta A

In [25]:
import os

base = ".."

df = pd.read_excel(os.path.join(base,'wines_dataset.xlsx'))
df_short = pd.read_excel(os.path.join('wines.xlsx'))

# find str in a column getting index
for name in df_short['Name']:
    index = df[df['Name'].str.contains(name, case=False)].index
    if len(index) > 0:
        print(index)
        df.loc[index,"Link"] = df_short[df_short['Name']==name]['Link'].values[0]
        
df.to_excel(os.path.join(base,'wines_dataset_link.xlsx'), index=False)

Index([309], dtype='int64')
Index([308], dtype='int64')
Index([305], dtype='int64')
Index([154], dtype='int64')
Index([168], dtype='int64')
Index([101], dtype='int64')
Index([343], dtype='int64')
Index([342], dtype='int64')
Index([549], dtype='int64')
Index([637], dtype='int64')
Index([445], dtype='int64')
Index([64, 65], dtype='int64')
Index([141], dtype='int64')
Index([307], dtype='int64')
Index([49, 50], dtype='int64')
Index([178], dtype='int64')
Index([150], dtype='int64')
Index([82, 83], dtype='int64')
Index([476], dtype='int64')
Index([336], dtype='int64')
Index([569, 570, 571, 572, 573, 574], dtype='int64')
Index([102, 103, 104, 105, 106, 107, 108], dtype='int64')
Index([57], dtype='int64')
Index([216], dtype='int64')
